In [ ]:
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install Bio

In [1]:
import pandas as pd
import os
import numpy as np
from Bio import Entrez

Entrez.email = 'gchiriaco@campus.unimib.it'

import xml.etree.ElementTree as ET
import os

In [2]:
data = pd.read_csv('sample_information_American_gut_microbiome.csv.txt', sep="\t", dtype=object)
data.head()

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,...,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.0,53.0,true,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,false,false,Not provided,40s,44.0,44.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,false,false,Not provided,60s,66.0,66.0,false,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided


Change all the true/false values to Yes/No

In [3]:
data[data=='true']='Yes'
data[data=='false']='No'
data[data=='I do not have this condition']='No'

In [4]:
list(data.columns)

['sample_name',
 'acid_reflux',
 'acne_medication',
 'acne_medication_otc',
 'add_adhd',
 'age_cat',
 'age_corrected',
 'age_years',
 'alcohol_consumption',
 'alcohol_frequency',
 'alcohol_types',
 'alcohol_types_beercider',
 'alcohol_types_red_wine',
 'alcohol_types_sour_beers',
 'alcohol_types_spiritshard_alcohol',
 'alcohol_types_unspecified',
 'alcohol_types_white_wine',
 'allergic_to',
 'allergic_to_i_have_no_food_allergies_that_i_know_of',
 'allergic_to_other',
 'allergic_to_peanuts',
 'allergic_to_shellfish',
 'allergic_to_tree_nuts',
 'allergic_to_unspecified',
 'altitude',
 'alzheimers',
 'animal_age',
 'animal_free_text',
 'animal_gender',
 'animal_origin',
 'animal_type',
 'anonymized_name',
 'antibiotic_history',
 'appendix_removed',
 'artificial_sweeteners',
 'asd',
 'assigned_from_geo',
 'autoimmune',
 'birth_year',
 'bmi',
 'bmi_cat',
 'bmi_corrected',
 'body_habitat',
 'body_product',
 'body_site',
 'bowel_movement_frequency',
 'bowel_movement_quality',
 'breastmilk_for

Selecting only the samples from feces

In [5]:

data = data.loc[data.body_site == "UBERON:feces",:]

Removing all the subjects with special diseases and conditions

In [6]:
cond_no_disease = ( (data['alcohol_frequency']!='Daily') &
                    ((data['autoimmune']=='No') | (data['autoimmune']=='Not provided')) &
                    ((data['cancer']=='No') | (data['cancer']=='Not provided')) &
                    ((data['cdiff']=='No') | (data['cdiff']=='Not provided')) & #Clostridioides difficile
                    ((data['diabetes']=='No') | (data['diabetes']=='Not provided')) &
                    ((data['ibd']=='No') | (data['ibd']=='Not provided')) & #inflammatory bowel disease
                    ((data['ibs']=='No') | (data['ibs']=='Not provided')) & #inflamatiry bowel syndrome
                    ((data['kidney_disease']=='No') | (data['kidney_disease']=='Not provided')) &
                    ((data['liver_disease']=='No') | (data['liver_disease']=='Not provided')) &
                    ((data['mental_illness_type_anorexia_nervosa']=='No') | (data['mental_illness_type_anorexia_nervosa']=='Not provided')) &
                    ((data['mental_illness_type_bulimia_nervosa']=='No') | (data['mental_illness_type_bulimia_nervosa']=='Not provided')) &
                    ((data['sibo']=='No') | (data['sibo']=='Not provided'))) #small intestinal bacterial overgrowth

data_no_disease = data.loc[cond_no_disease, :]

Number of subjects that removed refined sugars from their diet

In [7]:
data_no_disease.specialized_diet_exclude_refined_sugars.value_counts()

Not provided    7417
No              4633
Yes              633
Unspecified       22
Name: specialized_diet_exclude_refined_sugars, dtype: int64

Selecting only Male/Females and subject with age between 20s and 60s

In [8]:
data_no_disease.age_cat.value_counts()

30s             2359
40s             2288
50s             2221
60s             1735
20s             1352
Not provided     748
child            733
70+              594
teen             382
Unspecified      287
baby               6
Name: age_cat, dtype: int64

In [9]:
data_no_disease.sex.value_counts()

female          6178
male            5976
Not provided     526
unspecified       18
other              7
Name: sex, dtype: int64

In [10]:
data_f = data_no_disease.loc[((data_no_disease['age_cat'] == '20s') |
                          (data_no_disease['age_cat'] == '30s') |
                          (data_no_disease['age_cat'] == '40s') |
                          (data_no_disease['age_cat'] == '50s')|
                          (data_no_disease['age_cat'] == '60s'))  &
                          ((data_no_disease.sex == 'female') | (data_no_disease.sex == 'male')) ,:]

In [11]:
pd.crosstab(data_f.age_cat,
            data_f.specialized_diet_exclude_refined_sugars, 
            values=data_f.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Not provided,Unspecified,Yes,All
age_cat,,,,,
20s,573.0,698.0,NaN,59.0,1330
30s,869.0,1295.0,3.0,128.0,2295
40s,817.0,1330.0,3.0,105.0,2255
50s,771.0,1284.0,6.0,101.0,2162
60s,557.0,1026.0,4.0,92.0,1679
All,3587.0,5633.0,16.0,485.0,9721


In [12]:
pd.crosstab(data_f.sex,
            data_f.specialized_diet_exclude_refined_sugars, 
            values=data_f.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Not provided,Unspecified,Yes,All
sex,,,,,
female,1824,2945,12,250,5031
male,1763,2688,4,235,4690
All,3587,5633,16,485,9721


Creation of the Control/Treatment groups:
- Control: subjects without a particular health condition, that don't follow any specilized diet

- Treatment: subjects without a paritcular health condition, that exclude refined sugars from their diet and don't follow any other specilized diet

In [13]:
[x for x in list(data_f.columns) if x.lower().startswith("specialized_diet_")]

['specialized_diet_exclude_dairy',
 'specialized_diet_exclude_nightshades',
 'specialized_diet_exclude_refined_sugars',
 'specialized_diet_fodmap',
 'specialized_diet_halaal',
 'specialized_diet_i_do_not_eat_a_specialized_diet',
 'specialized_diet_kosher',
 'specialized_diet_modified_paleo_diet',
 'specialized_diet_other_restrictions_not_described_here',
 'specialized_diet_paleodiet_or_primal_diet',
 'specialized_diet_raw_food_diet',
 'specialized_diet_unspecified',
 'specialized_diet_westenprice_or_other_lowgrain_low_processed_fo',
 'specialized_diet_westenprice_or_other_lowgrain_low_processed_food_diet']

In [14]:
data_f= data_f.drop("specialized_diet_unspecified",axis=1)

Function that assigns every subject to a group

In [15]:
def get_groups(data):
    exclude_sugar = False
    other_diets = False
    specialized_diet = False

    list_col = list(data.index)
    list_col = [x for x in list_col if x.lower().startswith("specialized_diet_")]
    for col in list_col:
        if col =="specialized_diet_i_do_not_eat_a_specialized_diet":
            if data[col] == "No":
                specialized_diet = True
        elif col == "specialized_diet_exclude_refined_sugars":
            if data[col] == "Yes":
                exclude_sugar = True
        else:
            if data[col] == "Yes":
                other_diets = True
    
    if specialized_diet:
        
        if exclude_sugar and not other_diets:
            return "Treatment"
        else:
            return "None"
    
    else:
        if exclude_sugar or other_diets:
            return "None"
        
        return "Control"
        

In [16]:
data_f['group'] = data_f.apply(get_groups, axis=1)

Only 110 subjects have the characteristics to be in the Treatment group

In [17]:
data_f.group.value_counts()

Control      8089
None         1522
Treatment     110
Name: group, dtype: int64

Excluding all subjects that are neither in the control or treatment group

In [18]:
data_groups = data_f.loc[((data_f.group == "Control") | (data_f.group == "Treatment"))]

Sex distribution in the two groups

In [27]:
print(data_groups.groupby(["group","sex"]).sample_name.nunique())

group      sex   
Control    female    4159
           male      3930
Treatment  female      53
           male        57
Name: sample_name, dtype: int64


Sex and age distribution in the two groups

In [28]:
print(data_groups.groupby(["group","sex","age_cat"]).sample_name.nunique())

group      sex     age_cat
Control    female  20s        561
                   30s        925
                   40s        981
                   50s        957
                   60s        735
           male    20s        519
                   30s        943
                   40s        897
                   50s        885
                   60s        686
Treatment  female  20s          3
                   30s         14
                   40s         11
                   50s         12
                   60s         13
           male    20s         13
                   30s          9
                   40s         11
                   50s         12
                   60s         12
Name: sample_name, dtype: int64


In [28]:
data_groups.loc['age_years'] = data_groups.loc['age_years'].astype('float16')
print(data_groups.groupby(["group","sex"]).age_years.mean())

group      sex   
Control    female    45.37500
           male      45.28125
Treatment  female    48.62500
           male      44.90625
Name: age_years, dtype: float16


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


FastQ pull:

In [29]:
def get_fastq(sample_names, download = False, group = "treatment"):
    
    sample_name = list(sample_names)
    biosample_ids = list(sample_names)
    run_ids = list(sample_names) 
    i = 0
    
    for sample in sample_name:
        i+=1
        print(f"({i})\tGetting {sample_name[i-1]} Biosample ID")
        samplehandle = Entrez.esearch(db='biosample', term=sample)
        sampleread = Entrez.read(samplehandle)
                
        if len(sampleread['IdList']) == 0:
            print("No BiosampleID found")
            biosample_ids[i-1] = "Not found"
            run_ids[i-1] = "Not found"

        else: 
            print(f"\tFound {len(sampleread['IdList'])} BiosampleID: {sampleread['IdList']}")

            for bioid in sampleread['IdList']:
                try:
                    biohandle = Entrez.efetch(db='biosample', id=bioid, retmode='xml')
                    bioxml = ET.fromstring(biohandle.read())
                    sraids = bioxml.findall('.//BioSample//Ids//Id')
                    sraids = [x.text for x in sraids if x.attrib['db']=="SRA"]
                except:
                    run_ids[i-1] = "Not found"
                    continue 
                
                for sraid in sraids:
                    srahandle = Entrez.esearch(db='sra', term=sraid,retmode="xml")
                    srareads = Entrez.read(srahandle)["IdList"]
                    print(f"\tFound {len(srareads)} SRA ID: {srareads}")

                    sraidhandles =[Entrez.efetch(db='sra', id=s) for s in srareads]
                    root = [ET.fromstring(handle.read()) for handle in sraidhandles]
                    identifier = [r.find('.//EXPERIMENT_PACKAGE//RUN_SET//RUN') for r in root]
                    instrument = [r.find('.//EXPERIMENT_PACKAGE//EXPERIMENT//PLATFORM//ILLUMINA//INSTRUMENT_MODEL').text for r in root]

                    filt_id = [identifier[k] for k in range(len(identifier)) if instrument[k] == "Illumina MiSeq"]

                    if len(filt_id)>0 and filt_id[0] is not None:
                        one_id = filt_id[0].attrib["accession"]
                        print('\tSelected 1 Run ID:', one_id)
                        biosample_ids[i-1]=bioid
                        run_ids[i-1] = one_id
                    else:
                        print('\tNo RunID found')
                        run_ids[i-1] = "Not found"

                
                    # Download FASQ e FASTA dei RUNid
                    if download:
                        if not os.path.exists(f"fastq_{group}"):
                            os.mkdir(f"fastq_{group}")

                        print('\tDownloading FASTQ...')
                        command = f'sratoolkit.3.0.2-win64\\bin\\fastq-dump.exe --outdir fastq_{group}\\ --readids ' + one_id
                        os.system(command)


    dict = {'sample_names': sample_name, 
                'biosample_ids': biosample_ids,
                'run_ids': run_ids}  

    return dict    

Treatment group data

In [30]:
data_treatment = data_groups.loc[(data_groups.group == "Treatment")]

Control group data (selecting only 120 subjects)

In [31]:
data_control = data_groups.loc[(data_groups.group == "Control")]
data_control = data_control.groupby(["sex","age_cat"]).head(12)

Treatment group fastq pulling

In [33]:
dict_treatment = get_fastq(data_treatment.sample_name, download = False, group = 'treatment')

(1)	Getting 10317.000023139 Biosample ID
	Found 1 BiosampleID: ['7353729']
	Found 1 SRA ID: ['4279800']
	Selected 1 Run ID: ERR2032495
(2)	Getting 10317.000027811 Biosample ID
	Found 1 BiosampleID: ['7353735']
	Found 1 SRA ID: ['4279806']
	Selected 1 Run ID: ERR2032501
(3)	Getting 10317.000033280 Biosample ID
	Found 1 BiosampleID: ['8577201']
	Found 1 SRA ID: ['5144891']
	Selected 1 Run ID: ERR2313963
(4)	Getting 10317.000038261 Biosample ID
	Found 1 BiosampleID: ['8614208']
	Found 1 SRA ID: ['5162312']
	Selected 1 Run ID: ERR2318007
(5)	Getting 10317.000041075 Biosample ID
	Found 1 BiosampleID: ['8728556']
	Found 5 SRA ID: ['14635658', '14635351', '14634860', '14631050', '5245039']
	Selected 1 Run ID: ERR2404913
(6)	Getting 10317.000050189 Biosample ID
	Found 1 BiosampleID: ['6366098']
	Found 1 SRA ID: ['3736480']
	Selected 1 Run ID: ERR1842797
(7)	Getting 10317.000051222 Biosample ID
	Found 1 BiosampleID: ['9653436']
	Found 1 SRA ID: ['5960036']
	Selected 1 Run ID: ERR2696825
(8)	Get

Control group fastq pulling

In [35]:
dict_control = get_fastq(data_control.sample_name, download = False, group = 'control')

(1)	Getting 10317.000001001 Biosample ID
	Found 1 BiosampleID: ['4186901']
	Found 1 SRA ID: ['1912462']
	Selected 1 Run ID: ERR1073439
(2)	Getting 10317.000001002 Biosample ID
	Found 1 BiosampleID: ['4186120']
	Found 1 SRA ID: ['1911681']
	Selected 1 Run ID: ERR1072624
(3)	Getting 10317.000001004 Biosample ID
	Found 1 BiosampleID: ['4186121']
	Found 1 SRA ID: ['1911682']
	Selected 1 Run ID: ERR1072625
(4)	Getting 10317.000001008 Biosample ID
	Found 1 BiosampleID: ['4186122']
	Found 1 SRA ID: ['1911683']
	Selected 1 Run ID: ERR1072626
(5)	Getting 10317.000001018 Biosample ID
	Found 1 BiosampleID: ['4186123']
	Found 2 SRA ID: ['1955804', '1911684']
	Selected 1 Run ID: ERR1089664
(6)	Getting 10317.000001020 Biosample ID
	Found 1 BiosampleID: ['4188946']
	Found 1 SRA ID: ['1914607']
	Selected 1 Run ID: ERR1075599
(7)	Getting 10317.000001022 Biosample ID
	Found 1 BiosampleID: ['4187660']
	Found 1 SRA ID: ['1913249']
	Selected 1 Run ID: ERR1074147
(8)	Getting 10317.000001025 Biosample ID
	Fo

Id dataframe for both groups

In [36]:
treatment_fastq = pd.DataFrame.from_dict(dict_treatment)
control_fastq = pd.DataFrame.from_dict(dict_control)

Selecting only subjects for whom fastq was found

In [42]:
treatment_fastq = treatment_fastq.loc[treatment_fastq['run_ids'] != "Not found"].reset_index(drop= True)
control_fastq = control_fastq.loc[control_fastq['run_ids'] != "Not found"].reset_index(drop= True)

Selecting only 100 subjects for each group

In [47]:
data_control = data_control.loc[data_control['sample_name'].isin(control_fastq.sample_names)]
data_treatment = data_treatment.loc[data_treatment['sample_name'].isin(treatment_fastq.sample_names)]

In [57]:
data_control = data_control.groupby(["sex","age_cat"]).head(10).reset_index(drop=True)
data_treatment = data_treatment.head(100).reset_index(drop=True)

Concatenation of the two groups

In [60]:
data_final_groups = pd.concat([data_treatment,data_control]).reset_index(drop=True)

In [61]:
control_fastq = control_fastq.loc[control_fastq.sample_names.isin(data_control['sample_name'])]
treatment_fastq = treatment_fastq.loc[treatment_fastq.sample_names.isin(data_treatment['sample_name'])]

In [62]:
all_fastq = pd.concat([control_fastq,treatment_fastq]).reset_index(drop=True)

In [63]:
all_fastq

,sample_names,biosample_ids,run_ids
0,10317.000001001,4186901,ERR1073439
1,10317.000001002,4186120,ERR1072624
2,10317.000001004,4186121,ERR1072625
3,10317.000001008,4186122,ERR1072626
4,10317.000001018,4186123,ERR1089664
...,...,...,...
195,10317.000108903,18047274,ERR5329027
196,10317.000108906,18047334,ERR5329028
197,10317.000109791,18047300,ERR5329067
198,10317.000109807,18046629,ERR5332367


Saving the dataframe with all needed info

In [64]:
data_final_groups = pd.merge(data_final_groups,
                             all_fastq,
                             how = "inner",
                             left_on = "sample_name",
                             right_on="sample_names")
data_final_groups.to_csv("data_final_groups.csv")

# FASTQ - Quality check

In [12]:
from Bio import SeqIO
from os import listdir
from os.path import isfile, join
import os
import pandas as pd

In [7]:
def filter_quality(fastq_path,min_quality = 20,verbose = True):
    
    tot_reads = []
    saved_reads = []

    if not os.path.exists('good_quality_' + str(min_quality)):
        os.mkdir(f'good_quality_{str(min_quality)}')
    
    for f in os.listdir(fastq_path):
        print(f"Reading {f}")
        num_reads = 0 
        for rec in SeqIO.parse(os.path.join(fastq_path,f), "fastq"):
            num_reads +=1
            good_reads = (rec for rec in \
                            SeqIO.parse(os.path.join(fastq_path,f), "fastq") \
                                if min(rec.letter_annotations["phred_quality"]) >= min_quality)

        count = SeqIO.write(good_reads, f"good_quality_{str(min_quality)}/{f}", "fastq")
        print(f"Saved {count} reads")
        print(f"{num_reads} reads\n\n")
        tot_reads.append(num_reads)
        saved_reads.append(count)
    
    return os.listdir(fastq_path), tot_reads, saved_reads
    



In [15]:
names, tot_reads, good_reads = filter_quality("fastq")

Reading ERR2579789.fastq
Saved 822 reads
3583 reads


Reading ERR1072662.fastq
Saved 12741 reads
30155 reads


Reading ERR1077769.fastq
Saved 10081 reads
34965 reads


Reading ERR2318153.fastq
Saved 6909 reads
36283 reads


Reading ERR1074740.fastq
Saved 7831 reads
18542 reads


Reading ERR1074159.fastq
Saved 1 reads
2 reads


Reading ERR1072644.fastq
Saved 9925 reads
23118 reads


Reading ERR5326482.fastq
Saved 9406 reads
20175 reads


Reading ERR1072754.fastq
Saved 10460 reads
24932 reads


Reading ERR1072656.fastq
Saved 21361 reads
52425 reads


Reading ERR2404913.fastq
Saved 16881 reads
34268 reads


Reading ERR2314152.fastq
Saved 30183 reads
49733 reads


Reading ERR1074147.fastq
Saved 10093 reads
20900 reads


Reading ERR1074162.fastq
Saved 7112 reads
17906 reads


Reading ERR1072626.fastq
Saved 11106 reads
25136 reads


Reading ERR5332232.fastq
Saved 8168 reads
21965 reads


Reading ERR5328798.fastq
Saved 6696 reads
14172 reads


Reading ERR1074736.fastq
Saved 3455 reads
7736 re

In [18]:
good_reads_df = pd.DataFrame({"id":names,"tot_reads":tot_reads,"good_reads":good_reads})

In [20]:
good_reads_df.to_csv('good_reads.csv')